# Merging Instacart data sets

## Contents
## 1. Combine the orders_products_combined dataframe with products data set

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os

In [2]:
# Project folder path as string
path = r'/Users/sophie/Desktop/CareerFoundry /09 2023 Phython'
path

'/Users/sophie/Desktop/CareerFoundry /09 2023 Phython'

In [24]:
# Importing data sets 
orders_products_combined = pd.read_pickle(os.path.join(path, path, 'Data', 'prepared data ', 'orders_products_combined.pkl'))
orders_products_combined.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,True,196,1,0,both
1,2539329,1,1,2,8,NaN,True,14084,2,0,both
2,2539329,1,1,2,8,NaN,True,12427,3,0,both
3,2539329,1,1,2,8,NaN,True,26088,4,0,both
4,2539329,1,1,2,8,NaN,True,26405,5,0,both


In [14]:
# Checking shape of df
orders_products_combined.shape


(32434489, 11)

In [ ]:
# Compared to exported df: 32434489, 11 --> seems correct 

## 1. Combine the orders_products_combined dataframe with products data set
#### Using the merge function, because 2 df's should be combined on the same key in a wide format 

In [21]:
# Importing products data set 
prods = pd.read_csv(os.path.join(path, 'Data', 'prepared data ', 'products_checked.csv'), index_col = False)
prods.head()

,Unnamed: 0,product_id,product_name,aisle_id,department_id,prices
0,0,1,Chocolate Sandwich Cookies,61,19,5.8
1,1,2,All-Seasons Salt,104,13,9.3
2,2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,4,5,Green Chile Anytime Sauce,5,13,4.3


In [22]:
# Deleting unnamed column
df_prods = prods.drop(columns = ['Unnamed: 0'])
df_prods.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


In [23]:
df_prods.shape

(49672, 5)

In [ ]:
# Compared to exported dataframe: 49672, 5 --> looks correct

In [26]:
# Excluding _merge from combined data set, because we want to check merge flag in new combined data set 
orders_products_combined_del_merge = orders_products_combined.drop(columns = ['_merge'])
orders_products_combined_del_merge.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered
0,2539329,1,1,2,8,NaN,True,196,1,0
1,2539329,1,1,2,8,NaN,True,14084,2,0
2,2539329,1,1,2,8,NaN,True,12427,3,0
3,2539329,1,1,2,8,NaN,True,26088,4,0
4,2539329,1,1,2,8,NaN,True,26405,5,0


In [27]:
orders_products_combined_del_merge.shape

(32434489, 10)

In [30]:
# Merging data sets, checking with merge flag
# Indicator is product_id
df_merged_prods_comb = orders_products_combined_del_merge.merge(df_prods, on = 'product_id', indicator = True)
df_merged_prods_comb.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,new_customer,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,both
1,2398795,1,2,3,7,15.0,False,196,1,1,Soda,77,7,9.0,both
2,473747,1,3,3,12,21.0,False,196,1,1,Soda,77,7,9.0,both
3,2254736,1,4,4,7,29.0,False,196,1,1,Soda,77,7,9.0,both
4,431534,1,5,4,15,28.0,False,196,1,1,Soda,77,7,9.0,both


In [35]:
df_merged_prods_comb.shape

(32404859, 15)

In [ ]:
# 15 columns is correct, less rows than orders_products_combined_del_merge because of type = inner join?

In [32]:
# Confirm results using merge flag
df_merged_prods_comb['_merge'].value_counts()

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

In [41]:
# Double check with type=outer
test_df = orders_products_combined_del_merge.merge(df_prods, on = 'product_id', how = 'outer', indicator = True)
test_df['_merge'].value_counts()

_merge
both          32404859
left_only        30200
right_only          11
Name: count, dtype: int64

In [46]:
# Key column “product_id” does not exist completely in both dataframes, 30200 only from orders_products_combined_del_merge

# prods (49672, 5) + orders_products_combined_del_merge (32434489, 10) --> sum of all rows from both data sets
49672 + 32434489

32484161

In [47]:
sum_matching_keys = 32484161 -  30200
sum_matching_keys

# Why is this result not the count of "both"? --> duplicate keys

32453961

In [50]:
# Export data to pkl
df_merged_prods_comb.to_pickle(os.path.join(path, 'Data', 'prepared data ', 'orders_products_merged.pkl'))